In [ ]:
import numpy as np
import torch
import dgl
import time

In [5]:
RAWPATH = "/raid/bear/data/raw/papers100M"
nodeNUM = 111059956

GRAPHPATH = RAWPATH + "/graph.bin"
TRAINPATH = RAWPATH + "/trainIds.bin"

# for i in range(partNUM):
#     PATH = savePath + f"/part{i}" 
#     checkFilePath(PATH)

graph = torch.from_numpy(np.fromfile(GRAPHPATH,dtype=np.int32))
src,dst = graph[::2],graph[1::2]
trainIds = torch.from_numpy(np.fromfile(TRAINPATH,dtype=np.int64))
edgeTable = torch.zeros_like(src).to(torch.int32)
template_array = torch.zeros(nodeNUM,dtype=torch.int32,device="cuda")


In [ ]:
MAXEDGE = 1000000000
# Streaming edge data
batch_size = len(src) // MAXEDGE + 1
src_batches = torch.chunk(src, batch_size, dim=0)
dst_batches = torch.chunk(dst, batch_size, dim=0)
batch = [src_batches, dst_batches]


inNodeTable, outNodeTable = template_array.clone(), template_array.clone()
startTime = time.time()
for src_batch,dst_batch in zip(*batch):
    src_batch,dst_batch = src_batch.cuda(),dst_batch.cuda()
    inNodeTable,outNodeTable = dgl.sumDegree(inNodeTable,outNodeTable,src_batch,dst_batch)
src_batch,dst_batch = None,None
print(f"one loop using time :{time.time()-startTime:.3f}")
outNodeTable = outNodeTable.cpu() # innodeTable still in GPU for next use
src_batch,dst_batch = None,None

In [ ]:
outNodeTable